# Equity Volatility Model Comparison

This notebook demonstrates how to **compare different volatility models** using the **LSEG Analytics SDK**. 

By calculating multiple volatility surfaces with different model configurations, you can analyze how different mathematical approaches affect the resulting implied volatility structure. This is essential for model validation, risk analysis, and choosing appropriate models for specific use cases.

**What you'll learn:**
- How to calculate multiple volatility surfaces in a single request
- Comparing different volatility models (SVI vs SSVI)
- Visualizing differences between model outputs
- Batch processing multiple surface configurations


## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.market_data.eq_volatility` - Main ETI volatility surfaces SDK

This notebook uses external libraries **pandas, plotly**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.


In [1]:
from lseg_analytics.pricing.market_data import eq_volatility as ev

import plotly.graph_objects as go
import pandas as pd
import datetime as dt
import copy

## Data Preparation

For detailed explanation of fundamental steps to calculate volatility surfaces refer to **1_ETI_Volatility_Surfaces_Fundamentals** notebook.

### STEP 1: Create Surface Definition and Surface Parameters

In [2]:
print("Step 1: Creating Surface Definition...")

ric = "NVDA.O@RIC"

# Create surface definition object to specify the underlying instrument
surface_definition = ev.EtiSurfaceDefinition(
        instrument_code = ric
        )

print(f"   ✓ Instrument: {surface_definition.instrument_code}")

print("Step 2: Configuring Surface Parameters...")

# Create surface parameters object to define how the surface is calculated
surface_parameters = ev.EtiSurfaceParameters(
    # ------------- core surface params
    calculation_date = dt.datetime.strptime("2025-07-15", "%Y-%m-%d"),
    time_stamp = ev.CurvesAndSurfacesTimeStampEnum.DEFAULT,  # Options: CLOSE, OPEN, SETTLE, DEFAULT
    input_volatility_type = ev.InputVolatilityTypeEnum.IMPLIED,  # Options: IMPLIED, QUOTED
    volatility_model = ev.CurvesAndSurfacesVolatilityModelEnum.SSVI, # Options: SVI, SSVI

    # ----------------- SURFACE LAYOUT CONFIGURATION -------------------
    # Surface axis configuration for output layout
    x_axis = ev.XAxisEnum.STRIKE,  # Options: DATE, DELTA, MONEYNESS, STRIKE, TENOR
    y_axis = ev.YAxisEnum.DATE  # Options: same as X-axis
)

Step 1: Creating Surface Definition...
   ✓ Instrument: NVDA.O@RIC
Step 2: Configuring Surface Parameters...


In [3]:
# ============= MODEL COMPARISON SETUP =============

# Create a copy of the surface parameters for comparison
surface_parameters_compare = copy.deepcopy(surface_parameters)

# Change the volatility model for comparison
surface_parameters_compare.volatility_model = ev.CurvesAndSurfacesVolatilityModelEnum.SVI  # SVI model vs SSVI

print("Model Comparison Configuration:")
print(f"   ✓ Model 1: {surface_parameters.volatility_model.value}")
print(f"   ✓ Model 2: {surface_parameters_compare.volatility_model.value}")

Model Comparison Configuration:
   ✓ Model 1: SSVI
   ✓ Model 2: SVI


### STEP 2: Create Request Item and Layout

In [4]:
print("\n Step 3: Creating Request Item...")

# Create the main request object that combines all configuration
request_item = ev.EtiVolatilitySurfaceRequestItem(
    surface_tag = 'NVDA_Volsurface_SSVI',
    underlying_definition = surface_definition,
    surface_parameters = surface_parameters,
    underlying_type = ev.CurvesAndSurfacesUnderlyingTypeEnum.ETI
)

request_item_compare = copy.deepcopy(request_item)
request_item_compare.surface_tag = 'NVDA_Volsurface_SVI'
request_item_compare.surface_parameters = surface_parameters_compare

print(f"   ✓ Request Item: {request_item}")
print(f"   ✓ Request Item Compare: {request_item_compare}")


 Step 3: Creating Request Item...
   ✓ Request Item: {'surfaceTag': 'NVDA_Volsurface_SSVI', 'underlyingDefinition': {'instrumentCode': 'NVDA.O@RIC'}, 'surfaceParameters': {'calculationDate': '2025-07-15T00:00:00Z', 'timeStamp': 'Default', 'inputVolatilityType': 'Implied', 'volatilityModel': 'SSVI', 'xAxis': 'Strike', 'yAxis': 'Date'}, 'underlyingType': 'Eti'}
   ✓ Request Item Compare: {'surfaceTag': 'NVDA_Volsurface_SVI', 'underlyingDefinition': {'instrumentCode': 'NVDA.O@RIC'}, 'surfaceParameters': {'calculationDate': '2025-07-15T00:00:00Z', 'timeStamp': 'Default', 'inputVolatilityType': 'Implied', 'volatilityModel': 'SVI', 'xAxis': 'Strike', 'yAxis': 'Date'}, 'underlyingType': 'Eti'}


## Request Execution

In [5]:
# Execute the calculation using the calculate function
# The 'universe' parameter accepts a list of request items for batch processing
try:
    response = ev.calculate(universe=[request_item, request_item_compare])

    # Display response structure information
    surface_data = response['data'][0]
    if 'surface' in surface_data:
        print(f"   Calculation successful!")
        print(f"   Surface data points available: {len(surface_data['surface'])}")
    else:
        print("   No surface data found in response")
    
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise

   Calculation successful!
   Surface data points available: 20


## Results Display

In [6]:
# Plotting utils

def get_vol_surface_df(response, index=0):
    """
    Extract and format volatility surface data from API response into a DataFrame.
    
    This function processes the raw volatility surface data from an API response,
    handles various error conditions, and returns a properly formatted DataFrame
    suitable for visualization functions.

    """
    try:
        vol_surface = response['data'][index]['surface']
    except KeyError:
        print("No surface data available in response.")
        return None
    except TypeError:
        print("No surface data available in response.")
        return None

    expiries = vol_surface[0][1:]
    strikes = []
    values = []

    for row in vol_surface[1:]:
        strikes.append(row[0])
        values.append(row[1:])

    strikes = [round(float(s), 2) if isinstance(s, (int, float)) else s for s in strikes]

    surface_df = pd.DataFrame(values, index=strikes, columns=expiries).T
    surface_df = surface_df.astype(float).round(2)

    return surface_df

def plot_volatility_surface_plot(surf_table, x_axis, y_axis, colorscale="Turbo"):
    """
    Create an interactive 3D surface plot of the volatility surface with contour lines.
    
    This function generates a three-dimensional visualization where the x-axis represents
    moneyness/strikes, y-axis represents expiries, and z-axis represents volatility values.
    Contour lines are added for better depth perception.

    Parameters
    ----------
    surf_table : pd.DataFrame
        A DataFrame representing the volatility surface with expiries as index
        and strike prices/moneyness as columns. Values should be volatility levels.
    colorscale : str or list, optional
        Plotly colorscale name (e.g., 'Viridis', 'Turbo', 'Plasma') or custom colorscale list.
        Default is "Turbo".

    Returns
    -------
    plotly.graph_objects.Figure
  
    """
    if len(surf_table) < 2:
        fig = go.Figure()
        fig.add_annotation(
            text="Not enough data to display 3D surface",
            xref="paper", yref="paper",
            x=0.5, y=0.5,
            showarrow=False,
            font=dict(size=18),
            align="center"
        )
        fig.update_layout(
            title=dict(
                text="3D Surface Plot of Volatility Surface",
                x=0.5,
                xanchor="center",
                yanchor="top",
                y=0.95,
                font=dict(size=16)
            ),
            autosize=True,
            height=450,
            margin={"l": 0, "r": 0, "b": 0, "t": 50},
            dragmode=False,
        )
        return fig

    x = surf_table.columns
    y = surf_table.index
    z = surf_table.values

    fig = go.Figure(data=[go.Surface(
        z=z, 
        x=x, 
        y=y, 
        colorscale=colorscale,
        showscale=False,
        contours={
            "y": {
                "show": True,
                "color": "black",
                "highlightcolor": "black",
                "size": 0.05
            },
            "z": {
                "show": True,
                "color": "black",
                "highlightcolor": "black",
                "size": 0.05
            }
        }
    )])

    fig.update_layout(
        title=dict(
            text="3D Surface Plot of Volatility Surface",
            x=0.5,
            xanchor="center",
            yanchor="top",
            y=0.95,
            font=dict(size=16)
        ),
        scene={
            "xaxis_title": x_axis,
            "yaxis_title": y_axis,
            "zaxis_title": "Volatility",
            "xaxis": {"showgrid": True},
            "yaxis": {"showgrid": True},
            "zaxis": {"showgrid": True},
            "camera": {
                "eye": {"x": 0.96, "y": -1.53, "z": 0.39},
                "center": {"x": 0.02, "y": -0.07, "z": -0.21},
                "up": {"x": -0.18, "y": 0.27, "z": 0.95},
                "projection": {"type": "perspective"}
            }
        },
        dragmode=False,
        autosize=True,
        height=450,
        width=800,
    )

    return fig

def plot_surface_smile_by_expiry(surf_table, x_axis, y_axis):
    """
    Create a 2D line plot showing volatility smiles for different expiry dates.
    
    This function plots multiple volatility smile curves, with each curve representing
    a different expiry date. The x-axis shows moneyness/strikes and the y-axis shows
    volatility levels.

    Parameters
    ----------
    surf_table : pd.DataFrame
        A DataFrame with expiries as index and strike prices/moneyness as columns.
        Values should be volatility levels for each expiry-strike combination.

    Returns
    -------
    plotly.graph_objects.Figure
        Interactive line plot with each expiry represented as a separate trace.
        
    """
    fig = go.Figure()

    for expiry in surf_table.index:
        fig.add_trace(go.Scatter(
            x=surf_table.columns.astype(float),
            y=surf_table.loc[expiry],
            mode='lines+markers',
            name=expiry
        ))

    fig.update_layout(
        title=dict(
            text='Surface Smile by Expiry',
            x=0.5,
            xanchor="center",
            yanchor="top",
            y=0.95,
            font=dict(size=16)
        ),
        xaxis_title=x_axis,
        yaxis_title='Volatility',
        legend_title=y_axis,
        template='plotly_white',
        dragmode=False,
        autosize=True,
        height=450,
        width=800,
    )

    return fig

In [7]:
# Convert API response to pandas DataFrame for easier manipulation 
surface_df = get_vol_surface_df(response, index=0)           # SSVI model surface
surface_compare_df = get_vol_surface_df(response, index=1)   # SVI model surface

# Extract axis names for labeling plots
x_axis = surface_parameters.x_axis.name
y_axis = surface_parameters.y_axis.name

print("Surface Comparison Info:") 
print(f"   ✓ SSVI Surface: {surface_df.shape} (rows × columns)")
print(f"   ✓ SVI Surface: {surface_compare_df.shape} (rows × columns)") 
print(f"   ✓ x_axis: {x_axis}") 
print(f"   ✓ y_axis: {y_axis}") 

Surface Comparison Info:
   ✓ SSVI Surface: (15, 19) (rows × columns)
   ✓ SVI Surface: (13, 19) (rows × columns)
   ✓ x_axis: STRIKE
   ✓ y_axis: DATE


In [8]:
# Individual Surface Plots
print("SSVI Model Surface:")
plot_volatility_surface_plot(surface_df, x_axis, y_axis)

SSVI Model Surface:


In [9]:
print("SVI Model Surface:")
plot_volatility_surface_plot(surface_compare_df, x_axis, y_axis)

SVI Model Surface:


In [10]:
from IPython.display import display

# Difference Analysis
print("Model Difference (SSVI - SVI):")
diff_df = surface_df - surface_compare_df
display(diff_df)

Model Difference (SSVI - SVI):


,85.35,102.42,119.49,128.025,136.56,145.095,153.63,162.165,166.4325,170.7,174.9675,179.235,187.77,196.305,204.84,213.375,221.91,238.98,256.05
2025-08-15,8.77,4.46,1.42,0.62,0.43,0.90,1.89,3.04,3.52,3.85,4.00,3.95,3.43,2.59,1.71,0.91,0.23,-0.74,-1.33
2025-09-19,0.47,-1.03,-1.63,-1.60,-1.39,-1.11,-0.85,-0.72,-0.71,-0.72,-0.76,-0.82,-0.98,-1.13,-1.24,-1.31,-1.32,-1.20,-0.93
2025-10-17,-1.78,-1.93,-1.32,-0.79,-0.22,0.28,0.62,0.77,0.78,0.74,0.68,0.61,0.43,0.26,0.14,0.05,0.02,0.10,0.33
2025-11-21,-1.33,-1.31,-0.96,-0.75,-0.61,-0.53,-0.53,-0.57,-0.61,-0.64,-0.67,-0.69,-0.72,-0.70,-0.64,-0.54,-0.40,0.01,0.52
2025-12-19,-0.77,-0.68,-0.44,-0.34,-0.29,-0.29,-0.32,-0.36,-0.38,-0.40,-0.40,-0.41,-0.38,-0.32,-0.23,-0.08,0.09,0.54,1.07
2026-01-16,-0.30,0.00,0.34,0.45,0.50,0.49,0.45,0.40,0.38,0.36,0.34,0.33,0.35,0.39,0.47,0.58,0.73,1.09,1.56
2026-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2026-03-20,-0.40,-0.19,-0.06,-0.03,-0.05,-0.11,-0.16,-0.22,-0.26,-0.29,-0.32,-0.34,-0.37,-0.38,-0.37,-0.34,-0.29,-0.13,0.10
2026-05-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2026-06-18,-0.54,-0.33,-0.15,-0.09,-0.05,-0.04,-0.05,-0.07,-0.10,-0.12,-0.15,-0.18,-0.23,-0.30,-0.36,-0.43,-0.49,-0.61,-0.69
